In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from tqdm.notebook import tqdm
from colorist import Color
import time
import os

import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchsummary import summary
from torch.optim import Adam

from models.pyramid_ViG import DeepGCN
from timm.models import create_model
from timm.scheduler import CosineLRScheduler
from trainer import Trainer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
model = create_model('PyramidVIG_Tiny_GELU')
model = model.to(device)

<models.pyramid_ViG.PyramidVIG_Tiny_GELU.<locals>.OptInit object at 0x00000272BDD8E470>
using relative_pos
using relative_pos
using relative_pos
using relative_pos
using relative_pos
using relative_pos
using relative_pos
using relative_pos
using relative_pos
using relative_pos
using relative_pos
using relative_pos


In [3]:
rand_X = torch.randn(4, 3, 256, 256).to(device)
sample_output = model(rand_X)
print(sample_output.shape)

torch.Size([4, 7])


In [4]:
print(summary(model, (3, 256, 256)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 24, 128, 128]             672
       BatchNorm2d-2         [-1, 24, 128, 128]              48
              GELU-3         [-1, 24, 128, 128]               0
            Conv2d-4           [-1, 48, 64, 64]          10,416
       BatchNorm2d-5           [-1, 48, 64, 64]              96
              GELU-6           [-1, 48, 64, 64]               0
            Conv2d-7           [-1, 48, 64, 64]          20,784
       BatchNorm2d-8           [-1, 48, 64, 64]              96
              Stem-9           [-1, 48, 64, 64]               0
           Conv2d-10           [-1, 48, 64, 64]           2,352
      BatchNorm2d-11           [-1, 48, 64, 64]              96
     DenseDilated-12           [-1, 2, 4096, 9]               0
DenseDilatedKnnGraph-13           [-1, 2, 4096, 9]               0
           Conv2d-14          [-1, 9

In [5]:
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

valid_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

data_directory = './data'

train_dataset = ImageFolder(data_directory + '/train', transform=train_transform)
valid_dataset = ImageFolder(data_directory + '/valid', transform=valid_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, num_workers=4, pin_memory=True)

train_count = len(train_dataset)
valid_count = len(valid_dataset)
learning_rate = 2e-3
optimizer = Adam(model.parameters(), lr=learning_rate)
scheduler = CosineLRScheduler(optimizer, t_initial=20, warmup_lr_init=1e-6)

preparation_summary = {
    'data_directory': data_directory,
    'train_count': train_count,
    'valid_count': valid_count,
    'learning_rate': learning_rate,
    'class_names': [i for i in train_dataset.classes]
}

summary_df = pd.DataFrame(list(preparation_summary.values()),
                          index=preparation_summary.keys(),
                          columns=['Value'])
display(summary_df)

,Value
data_directory,./data
train_count,38569
valid_count,938
learning_rate,0.002
class_names,"[akiec, bcc, bkl, df, mel, nv, vasc]"


In [6]:
trainer = Trainer(model, train_loader, valid_loader, optimizer, scheduler, device)

Created new directory for model checkpoints at ./model_checkpoints/2024-03-11_04.37.57


In [7]:
trainer(epochs = 200)

Epoch 1:   0%|          | 0/1206 [00:00<?, ?it/s]

Epoch 1: 100%|██████████| 1206/1206 [07:21<00:00,  2.73it/s]


Train set ===> Average Loss: 0.0395 | Accuracy: 19889/38569 (51.57%)
Test set  ===> Average Loss: 0.0175 | Accuracy: 748/938 (79.74%)
Model saved at ./model_checkpoints/2024-03-11_04.37.57/best_model.pth
Best Accuracy: [79.744%]


Epoch 2: 100%|██████████| 1206/1206 [07:23<00:00,  2.72it/s]


Train set ===> Average Loss: 0.0320 | Accuracy: 23295/38569 (60.40%)
Test set  ===> Average Loss: 0.0144 | Accuracy: 786/938 (83.80%)
Model saved at ./model_checkpoints/2024-03-11_04.37.57/best_model.pth
Best Accuracy: [83.795%]


...

Epoch 14: 100%|██████████| 1206/1206 [07:21<00:00,  2.73it/s]


Train set ===> Average Loss: 0.0112 | Accuracy: 33378/38569 (86.54%)
Test set  ===> Average Loss: 0.0103 | Accuracy: 829/938 (88.38%)
Model saved at ./model_checkpoints/2024-03-11_04.37.57/best_model.pth
Best Accuracy: [88.38%]


Epoch 15: 100%|██████████| 1206/1206 [07:21<00:00,  2.73it/s]


Train set ===> Average Loss: 0.0102 | Accuracy: 33800/38569 (87.64%)
Test set  ===> Average Loss: 0.0113 | Accuracy: 830/938 (88.49%)
Model saved at ./model_checkpoints/2024-03-11_04.37.57/best_model.pth
Best Accuracy: [88.486%]


Epoch 16:  77%|███████▋  | 932/1206 [05:49<01:39,  2.75it/s]